<a href="https://colab.research.google.com/github/Hanbin-git/dacon_new_drug/blob/main/2025_07_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip -o "/content/drive/MyDrive/data.zip" -d "/content/data_1"


Archive:  /content/drive/MyDrive/data.zip
  inflating: /content/data_1/sample_submission.csv  
  inflating: /content/data_1/test.csv  
  inflating: /content/data_1/train.csv  


In [ ]:
# SMOTE 설치
!pip install -U imbalanced-learn
!pip install lightgbm optuna
!pip install -q lightgbm catboost xgboost



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 16.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade lightgbm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


In [ ]:
import lightgbm as lgb
print(lgb.__version__)

4.6.0


In [ ]:
# RDKit 설치 (Colab 전용)
!pip install rdkit-pypi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 14.1 MB/s eta 0:00:00


In [ ]:
# ✅ 기본 모듈 임포트
import os
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# ✅ 데이터 경로 설정
extract_path = "/content/data_1"

def get_path(filename):
    return os.path.join(extract_path, filename)

# ✅ 데이터 로딩
train_df = pd.read_csv(get_path("train.csv"))
test_df = pd.read_csv(get_path("test.csv"))
submission_df = pd.read_csv(get_path("sample_submission.csv"))

# ✅ 컬럼 공백 제거
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()
submission_df.columns = submission_df.columns.str.strip()

# ✅ Morgan Fingerprint 생성 함수
def get_morgan_fingerprint(smiles, n_bits=2048, radius=2):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        return np.array(fp)
    else:
        return np.zeros(n_bits)

# ✅ Feature 생성 (X, X_test)
train_fps = np.array([get_morgan_fingerprint(smi) for smi in train_df['Canonical_Smiles']])
test_fps = np.array([get_morgan_fingerprint(smi) for smi in test_df['Canonical_Smiles']])

X = pd.DataFrame(train_fps, columns=[f'fp_{i}' for i in range(train_fps.shape[1])])
X_test = pd.DataFrame(test_fps, columns=[f'fp_{i}' for i in range(test_fps.shape[1])])
y = train_df['Inhibition']

# ✅ 스케일링
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# ✅ 모델 학습 함수
def train_model(model, model_name):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(X_test))
    scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        oof_preds[val_idx] = preds
        rmse = np.sqrt(mean_squared_error(y_val, preds))  # ✅ 수정된 RMSE 계산
        scores.append(rmse)

        test_preds += model.predict(X_test) / kf.n_splits

        print(f"Fold {fold+1} RMSE: {rmse:.4f}")

    print(f"\n✅ {model_name} 평균 RMSE: {np.mean(scores):.4f}")
    return test_preds, oof_preds

# ✅ XGBoost / CatBoost 모델 정의
xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42, n_jobs=-1)
cat_model = CatBoostRegressor(iterations=500, learning_rate=0.05, depth=6, random_state=42, verbose=0)

# ✅ 모델 학습 및 예측
xgb_test_preds, _ = train_model(xgb_model, "XGBoost")
cat_test_preds, _ = train_model(cat_model, "CatBoost")

# ✅ 결과 저장
submission_df['XGB_Pred'] = np.clip(xgb_test_preds, 0, 100)
submission_df['CAT_Pred'] = np.clip(cat_test_preds, 0, 100)
submission_df.to_csv("submission_xgb_cat.csv", index=False)

# ✅ 결과 확인
submission_df.head()


Fold 1 RMSE: 25.5308
Fold 2 RMSE: 24.5531
Fold 3 RMSE: 27.2464
Fold 4 RMSE: 25.6300
Fold 5 RMSE: 26.2543

✅ XGBoost 평균 RMSE: 25.8429
Fold 1 RMSE: 24.6095
Fold 2 RMSE: 23.7765
Fold 3 RMSE: 26.0818
Fold 4 RMSE: 24.8891
Fold 5 RMSE: 25.4156

✅ CatBoost 평균 RMSE: 24.9545


,ID,Inhibition,XGB_Pred,CAT_Pred
0,TEST_000,0,28.732483,29.473925
1,TEST_001,0,26.949112,32.560237
2,TEST_002,0,32.170065,27.581673
3,TEST_003,0,33.437640,41.026182
4,TEST_004,0,23.882784,24.906842


In [ ]:
# ✅ 라이브러리 불러오기
import os
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# ✅ 경로 설정 및 데이터 로드
extract_path = "/content/data_1"
def get_path(filename): return os.path.join(extract_path, filename)

train_df = pd.read_csv(get_path("train.csv"))
test_df = pd.read_csv(get_path("test.csv"))
submission_df = pd.read_csv(get_path("sample_submission.csv"))

train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()
submission_df.columns = submission_df.columns.str.strip()

# ✅ Morgan Fingerprint 생성
def get_morgan_fingerprint(smiles, n_bits=2048, radius=2):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits))
    else:
        return np.zeros(n_bits)

train_fps = np.array([get_morgan_fingerprint(smi) for smi in train_df['Canonical_Smiles']])
test_fps = np.array([get_morgan_fingerprint(smi) for smi in test_df['Canonical_Smiles']])

X = pd.DataFrame(train_fps, columns=[f'fp_{i}' for i in range(train_fps.shape[1])])
X_test = pd.DataFrame(test_fps, columns=[f'fp_{i}' for i in range(test_fps.shape[1])])
y = train_df['Inhibition']

# ✅ 스케일링
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# ✅ 모델 학습 함수
def train_model(model, model_name):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(X_test))
    scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        oof_preds[val_idx] = preds
        fold_rmse = np.sqrt(mean_squared_error(y_val, preds))
        scores.append(fold_rmse)

        test_preds += model.predict(X_test) / kf.n_splits

    print(f"\n✅ {model_name} 평균 RMSE: {np.mean(scores):.4f}")
    return test_preds, oof_preds

# ✅ 기본 모델 정의
xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42, n_jobs=-1)
cat_model = CatBoostRegressor(iterations=500, learning_rate=0.05, depth=6, random_state=42, verbose=0)

# ✅ XGB & CAT 학습
xgb_test_preds, xgb_oof = train_model(xgb_model, "XGBoost")
cat_test_preds, cat_oof = train_model(cat_model, "CatBoost")

# ✅ Soft Voting
soft_voting_preds = (xgb_test_preds + cat_test_preds) / 2

# ✅ Stacking (Ridge 메타 모델)
stacked_X = np.vstack([xgb_oof, cat_oof]).T
stacked_test = np.vstack([xgb_test_preds, cat_test_preds]).T

meta_model = Ridge()
meta_model.fit(stacked_X, y)
stacked_preds = meta_model.predict(stacked_test)

# ✅ 결과 저장
submission_df['XGB_Pred'] = np.clip(xgb_test_preds, 0, 100)
submission_df['CAT_Pred'] = np.clip(cat_test_preds, 0, 100)
submission_df['Soft_Voting'] = np.clip(soft_voting_preds, 0, 100)
submission_df['Stacked_Pred'] = np.clip(stacked_preds, 0, 100)
submission_df["Inhibition"] = submission_df["Stacked_Pred"]

submission_df.to_csv("submission_xgb_cat_soft_stacked.csv", index=False)
submission_df.head()



✅ XGBoost 평균 RMSE: 25.8429

✅ CatBoost 평균 RMSE: 24.9545


,ID,Inhibition,XGB_Pred,CAT_Pred,Soft_Voting,Stacked_Pred
0,TEST_000,29.950850,28.732483,29.473925,29.103204,29.950850
1,TEST_001,32.357282,26.949112,32.560237,29.754675,32.357282
2,TEST_002,28.605795,32.170065,27.581673,29.875869,28.605795
3,TEST_003,39.591117,33.437640,41.026182,37.231911,39.591117
4,TEST_004,25.973411,23.882784,24.906842,24.394813,25.973411


In [ ]:
# 최종 제출 파일 생성
submission_df[["ID", "Stacked_Pred"]].rename(columns={"Stacked_Pred": "Inhibition"}).to_csv("submission.csv", index=False)


In [ ]:
# ✅ 라이브러리 임포트
import os
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys, Descriptors
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# ✅ 경로 및 데이터 로드
extract_path = "/content/data_1"
def get_path(filename): return os.path.join(extract_path, filename)

train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

# ✅ Feature 추출 함수
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(2048 + 167 + 6)

    # Morgan
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    morgan_arr = np.array(morgan_fp)

    # MACCS
    maccs_fp = MACCSkeys.GenMACCSKeys(mol)
    maccs_arr = np.array(maccs_fp)

    # Descriptors
    desc = [
        Descriptors.MolWt(mol),
        Descriptors.TPSA(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumHAcceptors(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.MolLogP(mol)
    ]

    return np.concatenate([morgan_arr, maccs_arr, desc])

# ✅ Feature Engineering
X = np.array([extract_features(smi) for smi in train["Canonical_Smiles"]])
X_test = np.array([extract_features(smi) for smi in test["Canonical_Smiles"]])
y = train["Inhibition"]

# ✅ Scaling & PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=100)
X_reduced = pca.fit_transform(X_scaled)
X_test_reduced = pca.transform(X_test_scaled)

# ✅ 모델 학습 함수
def train_model(model, name):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    oof_preds = np.zeros(len(X_reduced))
    test_preds = np.zeros(len(X_test_reduced))
    scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_reduced)):
        X_train, X_val = X_reduced[train_idx], X_reduced[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        oof_preds[val_idx] = preds

        # ✅ RMSE 수동 계산 (버전 문제 방지)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        scores.append(rmse)
        print(f"{name} Fold {fold+1} RMSE: {rmse:.4f}")

        test_preds += model.predict(X_test_reduced) / kf.n_splits

    print(f"\n✅ {name} 평균 RMSE: {np.mean(scores):.4f}")
    return test_preds, oof_preds

# ✅ 모델 정의
xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42, n_jobs=-1)
cat_model = CatBoostRegressor(iterations=500, learning_rate=0.05, depth=6, random_state=42, verbose=0)

# ✅ 모델 학습 및 예측
xgb_test_preds, _ = train_model(xgb_model, "XGBoost")
cat_test_preds, _ = train_model(cat_model, "CatBoost")

# ✅ 앙상블
final_preds = (xgb_test_preds + cat_test_preds) / 2

# ✅ 결과 저장
submission["Inhibition"] = np.clip(final_preds, 0, 100)
submission[["ID", "Inhibition"]].to_csv("submission_final.csv", index=False)


XGBoost Fold 1 RMSE: 24.9776
XGBoost Fold 2 RMSE: 23.6717
XGBoost Fold 3 RMSE: 25.9837
XGBoost Fold 4 RMSE: 24.8330
XGBoost Fold 5 RMSE: 25.1021

✅ XGBoost 평균 RMSE: 24.9136
CatBoost Fold 1 RMSE: 24.3470
CatBoost Fold 2 RMSE: 23.9257
CatBoost Fold 3 RMSE: 25.8010
CatBoost Fold 4 RMSE: 24.8887
CatBoost Fold 5 RMSE: 25.1095

✅ CatBoost 평균 RMSE: 24.8144


In [ ]:
pip install numpy==1.23.5


In [ ]:
# ✅ 라이브러리
import pandas as pd, numpy as np, os
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, MACCSkeys
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
import optuna
import warnings; warnings.filterwarnings("ignore")

# ✅ 경로 설정 및 데이터 로드
extract_path = "/content/data_1"
def get_path(filename): return os.path.join(extract_path, filename)
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

# ✅ RDKit 특성 추출 함수
def featurize(smiles_list):
    features = []
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol:
            features.append([
                Descriptors.MolWt(mol),
                Descriptors.MolLogP(mol),
                Descriptors.TPSA(mol),
                Descriptors.NumHDonors(mol),
                Descriptors.NumHAcceptors(mol),
                Descriptors.NumRotatableBonds(mol),
                Descriptors.HeavyAtomCount(mol),
                Descriptors.FractionCSP3(mol),
                Descriptors.RingCount(mol),
                Descriptors.BalabanJ(mol)
            ])
        else:
            features.append([0]*10)
    return np.array(features)

# ✅ Feature Engineering
X = featurize(train["Canonical_Smiles"])
X_test = featurize(test["Canonical_Smiles"])
y = train["Inhibition"]

# ✅ Optuna를 이용한 XGBoost 튜닝
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 10),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 10),
        "random_state": 42
    }
    scores = []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for tr_idx, val_idx in kf.split(X):
        model = XGBRegressor(**params)
        model.fit(X[tr_idx], y[tr_idx])
        preds = model.predict(X[val_idx])
        mse = mean_squared_error(y[val_idx], preds)  # ✅ y[val_idx]로 직접 지정
        rmse = np.sqrt(mse)
        scores.append(rmse)
    return np.mean(scores)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)
best_params = study.best_params
best_params["random_state"] = 42

# ✅ 모델 정의
xgb_model = XGBRegressor(**best_params)
cat_model = CatBoostRegressor(verbose=0, random_state=42)
stacking_model = StackingRegressor(
    estimators=[("xgb", xgb_model), ("cat", cat_model)],
    final_estimator=Ridge(alpha=1.0),
    n_jobs=-1
)

# ✅ 학습 및 예측
stack_preds = np.zeros(len(X_test))
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for tr_idx, val_idx in kf.split(X):
    stacking_model.fit(X[tr_idx], y[tr_idx])
    stack_preds += stacking_model.predict(X_test) / 5

# ✅ 결과 저장
submission["Inhibition"] = np.clip(stack_preds, 0, 100)
submission.to_csv("submission_stacking_optuna.csv", index=False)


[I 2025-07-01 02:10:43,340] A new study created in memory with name: no-name-635e4efd-8e18-4f23-96bf-7ab3598d7cb4
[I 2025-07-01 02:10:56,572] Trial 0 finished with value: 25.99775935158504 and parameters: {'n_estimators': 792, 'max_depth': 8, 'learning_rate': 0.09340543242594898, 'subsample': 0.829139865796215, 'colsample_bytree': 0.6068572648992069, 'reg_alpha': 0.5406615367777556, 'reg_lambda': 7.173507213947854}. Best is trial 0 with value: 25.99775935158504.
[I 2025-07-01 02:11:08,988] Trial 1 finished with value: 26.149917930556256 and parameters: {'n_estimators': 885, 'max_depth': 8, 'learning_rate': 0.11157620463420154, 'subsample': 0.9358295374904899, 'colsample_bytree': 0.8048075359448115, 'reg_alpha': 9.310649031083386, 'reg_lambda': 8.675421524286673}. Best is trial 0 with value: 25.99775935158504.
[I 2025-07-01 02:11:10,665] Trial 2 finished with value: 26.945794649297067 and parameters: {'n_estimators': 812, 'max_depth': 3, 'learning_rate': 0.08692221219397638, 'subsample'

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import KFold
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import warnings

warnings.filterwarnings("ignore")

# ✅ 경로 설정 및 데이터 로드
extract_path = "/content/data_1"
def get_path(filename): return os.path.join(extract_path, filename)
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

# ✅ Morgan Fingerprint 변환 함수
def mol_to_morgan_fp(smiles, radius=2, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros((n_bits,))
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)

X = np.array([mol_to_morgan_fp(s) for s in train["Canonical_Smiles"]])
X_test = np.array([mol_to_morgan_fp(s) for s in test["Canonical_Smiles"]])
y = np.log1p(train["Inhibition"].values)  # ✅ Target log1p 변환

# ✅ 모델 정의
xgb = XGBRegressor(random_state=42, n_estimators=300, max_depth=6, learning_rate=0.05)
cat = CatBoostRegressor(random_state=42, verbose=0, iterations=300, depth=6, learning_rate=0.05)
meta_model = Ridge()

model = StackingRegressor(
    estimators=[("xgb", xgb), ("cat", cat)],
    final_estimator=meta_model,
    cv=5
)

# ✅ KFold 교차검증
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))

for train_idx, val_idx in kf.split(X):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    model.fit(X_train, y_train)
    oof_preds[val_idx] = model.predict(X_val)
    test_preds += model.predict(X_test) / kf.n_splits

# ✅ 평가 (역변환 후 RMSE)
rmse = np.sqrt(mean_squared_error(np.expm1(y), np.expm1(oof_preds)))
print(f"RMSE (log1p 적용 후 역변환): {rmse:.5f}")

# ✅ 제출 파일 생성
submission["Inhibition"] = np.expm1(test_preds)
submission.to_csv("submission_log1p_stacked.csv", index=False)


RMSE (log1p 적용 후 역변환): 29.14311


In [ ]:
# ✅ 예측 결과와 실제값 비교
error_df = pd.DataFrame({
    "ID": train["ID"],
    "Canonical_Smiles": train["Canonical_Smiles"],
    "True": y,
    "Pred": oof_preds,
})
error_df["Abs_Error"] = (error_df["True"] - error_df["Pred"]).abs()

# ✅ 상위 10개 오류 큰 샘플 확인
worst_df = error_df.sort_values("Abs_Error", ascending=False).head(50)
print(worst_df[["ID", "Canonical_Smiles", "True", "Pred", "Abs_Error"]])


              ID                                   Canonical_Smiles      True  \
595   TRAIN_0595                   O=S1(=O)N=C(NCCc2cccs2)c3ccccc13  0.000000   
578   TRAIN_0578                         Clc1cccnc1n2ccc(n2)c3nccs3  0.000000   
565   TRAIN_0565                   Cc1ccc2Oc3ccc(Cl)cc3C=C(C#N)c2c1  0.000000   
288   TRAIN_0288                 COc1cc2NC(=O)C=C(c3ccccc3F)c2cc1OC  0.000000   
499   TRAIN_0499      CC1=C(c2ccc(Cl)cc2)S(=O)(=O)N=C1c3cc4ccccc4s3  0.000000   
337   TRAIN_0337          CCCC(C(=O)O)c1c(C)ncnc1c2cc(F)c3OCCCc3c2C  0.000000   
440   TRAIN_0440         CCOC(=O)C1=C(Cc2ccccc2O1)n3cc(nn3)c4ccccc4  0.000000   
274   TRAIN_0274               COc1nc(N)nc2N(Cc3cccnc3)C(=O)N(C)c12  0.000000   
323   TRAIN_0323       Cc1oc(nc1COc2cccc(c2)C3CCCCN3CC(=O)O)c4cccs4  0.000000   
582   TRAIN_0582                        COc1ccc(Oc2cc(NCCO)ncn2)cc1  0.000000   
454   TRAIN_0454                   CCCNC(=O)c1cnn2ccc(nc12)c3ccccc3  0.000000   
244   TRAIN_0244            

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor

# ✅ 경로 설정 및 데이터 로드
extract_path = "/content/data_1"
def get_path(filename): return os.path.join(extract_path, filename)

train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

# ✅ 로그 변환된 타겟
train['Inhibition_log'] = np.log1p(train['Inhibition'])

# ✅ 간단한 피처 생성
def featurize(smiles):
    return {
        'length': len(smiles),
        'num_C': smiles.count('C'),
        'num_N': smiles.count('N'),
        'num_O': smiles.count('O'),
        'num_S': smiles.count('S'),
        'num_cl': smiles.lower().count('cl'),
    }

X = pd.DataFrame([featurize(s) for s in train['Canonical_Smiles']])
X_test = pd.DataFrame([featurize(s) for s in test['Canonical_Smiles']])
y = train['Inhibition_log'].values

# ✅ 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# ✅ 모델 학습: LightGBM + Quantile Loss
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
    X_tr, y_tr = X[tr_idx], y[tr_idx]
    X_val, y_val = X[val_idx], y[val_idx]

    model = LGBMRegressor(
        objective='quantile',
        alpha=0.5,
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        random_state=fold,
        verbosity=-1
    )

    model.fit(X_tr, y_tr)

    oof_preds[val_idx] = model.predict(X_val)
    test_preds += model.predict(X_test) / kf.n_splits

# ✅ 평가 및 저장
rmse = np.sqrt(mean_squared_error(np.expm1(y), np.expm1(oof_preds)))
print(f"✅ RMSE (Quantile Loss + log1p 역변환): {rmse:.5f}")

# ✅ 제출 파일 생성
submission['Inhibition'] = np.expm1(test_preds)
submission.to_csv("submission_quantile_lgbm.csv", index=False)
print("🎯 'submission_quantile_lgbm.csv' 저장 완료")


✅ RMSE (Quantile Loss + log1p 역변환): 27.27602
🎯 'submission_quantile_lgbm.csv' 저장 완료
